# 量子金融应用：投资组合分散化

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概览

当前量子计算应用到金融问题上的解决方案通常可分为三类量子算法，即量子模拟，量子优化以及量子机器学习 [1,2]。许多的金融问题本质上是一个组合优化问题，解决这些问题的算法通常具有较高的时间复杂度，实现难度较大。得益于量子计算强大的计算性能，未来有望通过量子算法解决这些复杂问题。

量桨的 Quantum Finance 模块主要讨论的是量子优化部分的内容，即如何通过一些量子算法解决实际金融应用中的优化问题。本文主要介绍如何使用量子算法求解被动投资管理中投资组合分散化问题。

## 投资组合分散化问题

普通用户受专业知识和市场经验的不足的限制，在实际的投资中偏向于被动投资策略。指数投资就是一种常见的被动投资例子，比如说投资者长期购买并持有标普 $500$ 指数（Standard & Poor’s $500$）。作为投资人，假如你不想投资已有的指数，那么你也可以自己创建特定的指数投资组合，在市场中挑选合适的股票加入到创建的指数投资组合中。

分散化是投资组合中平衡风险和收益的一个重要方法。对投资组合分散化的一个具体描述如下：当前可投资的股票数量为 $n$，指数投资组合中包含的股票数量为 $K$，需要对这 $n$ 个股票进行聚类，根据相似性将可选的股票划分为 $K$ 个类别，再从每个类别中选出最能代表该类别的股票，将其加入到指数组合中来，便于更好的控制风险，提高收益。

### 编码投资组合分散化问题

为了将投资组合分散化问题转化成一个参数化量子电路（parameterized quantum circuits, PQC）可解的问题，我们首先需要编码该问题的哈密顿量。

在对该问题进行建模时，需要明确的有两个问题，第一是如何对不同的股票进行分类，第二便是以什么样的标准挑选代表性的股票。为了解决这两个问题，首先需要定义股票 $i$ 和股票 $j$ 之间的相似度 $\rho_{ij}$:
* $\rho_{ii} = 1 \quad $ 该股票和其自身的相似度为1
* $\rho_{ij} \leq 1 \quad$  不同股票间 $\rho_{ij}$ 越大，相似度越高

由于两股票间收益率的相关性，我们可以在协方差矩阵基础上进一步对时间序列间的相似性进行度量。动态时间规整（Dynamic Time Warping, DTW）是一种常见的衡量两个时间序列之间相似度的方法，在本文中，采用DTW算法来计算两股票之间的相似性。基于该度量，我们可以对股票进行分类并挑选代表性股票。对于给定的 $n$ 支股票，每支股票我们可以定义 $n$ 个二进制变量 $x_{ij}$ 和 $1$ 个二进制变量 $y_j$。对于变量 $x_{ij}$，每 $n$ 位一组，$i$ 表示是第几支股票，$j$ 表示在该股票对应的 $n$ 个二进制变量中的序号。每支股票的 $n$ 位二进制变量如果相同位置为 $1$ (即 $j$ 相同)，则说明这两只股票被分为同一类，其中 $i = j$ 的就是该类别中被选到指数组合中的最具代表性的股票：

$$
x_{ij}=
\begin{cases}
1, & \text{指数组合中的股票 $j$ 和股票 $i$ 具有最高的相似度}\\
0, & \text{其他情况}
\end{cases},
$$

$$
y_{j}=
\begin{cases}
1, & \text{某类中的代表性股票 $j$ 被选择到指数组合中}\\
0, & \text{其他情况}
\end{cases}.
$$

在该问题中我们的模型便可以写作：

$$
\mathcal{M}= \max_{x_{ij}}\sum_{i=1}^n\sum_{j=1}^n \rho_{ij}x_{ij}. \tag{1}
$$

该模型需要满足以下几类约束：
* 聚类约束：限制指数组合中只能有 $K$ 支股票
    - $ \sum_{j=1}^n y_j = K$
* 整数约束：限制一只股票要么是在指数组合中，要么就不在
     - $ x_{ij},y_j\in{\{0,1\}}, \forall i = 1, \dots,n; j = 1, \dots, n$
* 一致性约束：保证如果一只股票可以代表另一支股票，那么它必须在指数组合中
    - $\sum_{j=1}^n x_{ij} = 1, \forall i = 1,\dots,n$
    - $x_{ij} \leq y_j, \forall i = 1,\dots,n; j = 1,\dots, n$
    - $x_{jj} = y_j, \forall j = 1,\dots,n$

该模型目标就是让可选择的 $n$ 个股票与挑选的指数股票组合间相似性最大化。

由于要对代价函数做梯度下降优化，所以在定义时就根据模型方程和相应的约束条件做一定修改：

$$
\begin{aligned}
C_x &= -\sum_{i=1}^{n}\sum_{j=1}^{n}\rho_{ij}x_{ij} + A\left(K- \sum_{j=1}^n y_j \right)^2 + \sum_{i=1}^n A\left(\sum_{j=1}^n 1- x_{ij} \right)^2 \\
&\quad + \sum_{j=1}^n A\left(x_{jj} - y_j\right)^2 + \sum_{i=1}^n \sum_{j=1}^n A\left(x_{ij}(1 - y_j)\right).\\ 
\end{aligned} \tag{2}
$$ 

该式子中第一项为相似性最大化，后面四项均为约束条件，$A$ 为惩罚参数，通常设置为较大的数字，使得最终表示指数投资组合结果的二进制字符串满足约束条件。

现在我们需要将代价函数转为一个哈密顿量，从而完成投资组合分散化问题的编码。每一个二进制变量可以取0和1两个值，分别对应量子态 $|0\rangle$ 和 $|1\rangle$。每个二进制变量都对应一个量子比特，所以我们需要 $n^2 + n$ 个量子比特来解决投资组合分散化问题。因为我们的变量 $x_{ij}$ 的值为 $0$ 和 $1$，所以我们要构造一个本征值和它对应的哈密顿量。泡利 $Z$ 的本征值为 $\pm 1$，于是我们构造的哈密顿量为 $\frac{I-Z}{2}$，对应的本征值即为 $0$ 和 $1$。

我们现在将二进制变量映射到泡利 $Z$ 矩阵上，从而使 $C_x$ 转化成哈密顿矩阵：

$$
x_{ij} \mapsto \frac{I-Z_{ij}}{2}, \tag{3}
$$

这里 $Z_{ij} = I \otimes I \otimes \ldots \otimes Z \otimes \ldots \otimes I$，也就是说 $Z$ 作用在 $ij$ 的量子比特上。通过这个映射，如果一个编号为 $ij$ 的量子比特的量子态为 $|1\rangle$，那么对应的二进制变量的取值为 $x_{ij} |1\rangle = \frac{I-Z_{ij}}{2} |1\rangle = 1|1\rangle $，也就是说该项目是我们要投资的。同样地，对于量子态为 $|0\rangle$的量子比特 $i$，它所对应的二进制变量的取值为 $x_{ij}|0\rangle  = \frac{I-Z_{ij}}{2} |0\rangle = 0 |0\rangle $。

我们用上述映射将 $C_x$ 转化成量子比特数为 $n^2+n$ 的系统的哈密顿矩阵 $H_C$（其中 $x_{ij}$ 占 $n^2$ 个qubit，$y_j$ 占 $n$ 个 qubit），从而实现了投资组合分散化问题的量子化。这个哈密顿矩阵 $H_C$ 的基态即为投资组合分散化问题的最优解。在接下来的部分，我们将展示如何用参数化量子电路找到这个矩阵的基态，也就是对应最小本征值的本征态。

## Paddle Quantum 实现

要在量桨上实现用参数化量子电路解决量子金融中的投资组合分散化问题，首先要做的便是加载需要用到的包。

In [1]:
#加载额外需要的包
import numpy as np
import pandas as pd
import datetime

#加载飞桨，量桨相关的模块
import paddle
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.finance import DataSimulator
from paddle_quantum.finance import portfolio_diversification_hamiltonian

### 准备实验数据

和投资组合优化问题相似，在本问题中，我们选定的投资项目类型为股票。对于实验测试要用的数据，提供了两种方法：
* 第一种是根据一定的条件，随机生成数据。

如果采用这种方法准备实验数据，用户在初始化数据时，就需要给出可投资股票的名字列表，交易数据的开始日期和结束日期。

In [2]:
num_assets = 3 #可选择股票数目
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = DataSimulator( stocks = stocks, start = datetime.datetime(2016, 1, 1), end = datetime.datetime(2016, 1, 30))  # 根据指定的条件，随机生成测试所需的数据

* 第二种是用户可以选择读取本地收集到的真实数据集用于实验。考虑到文件中包含的股票数可能会很多，用户可以指定用于该实验的股票数量，即 上面初始化的`num_assets`。

我们收集了 $12$ 支股票 $35$ 个交易日的收盘价格存放到 `realStockData_12.csv` 文件中，在这里我们只选择读取前 $3$ 个股票的信息。

在本教程中，我们选择读取的真实数据作为实验数据。

In [3]:
df = pd.read_csv('realStockData_12.csv')
dt = []
for i in range(num_assets):
    mylist = df['closePrice'+str(i)].tolist()
    dt.append(mylist)
print(dt) # 输出从文件中读取的3个股票在35个交易日中的收盘价格

data.set_data(dt) # 指定实验数据为用户读取的数据

[[16.87, 17.18, 17.07, 17.15, 16.66, 16.79, 16.69, 16.99, 16.76, 16.52, 16.33, 16.39, 16.45, 16.0, 16.09, 15.54, 13.99, 14.6, 14.63, 14.77, 14.62, 14.5, 14.79, 14.77, 14.65, 15.03, 15.37, 15.2, 15.24, 15.59, 15.58, 15.23, 15.04, 14.99, 15.11, 14.5], [32.56, 32.05, 31.51, 31.76, 31.68, 32.2, 31.46, 31.68, 31.39, 30.49, 30.53, 30.46, 29.87, 29.21, 30.11, 28.98, 26.63, 27.62, 27.64, 27.9, 27.5, 28.67, 29.08, 29.08, 29.95, 30.8, 30.42, 29.7, 29.65, 29.85, 29.25, 28.9, 29.33, 30.11, 29.67, 29.59], [5.4, 5.48, 5.46, 5.49, 5.39, 5.47, 5.46, 5.53, 5.5, 5.47, 5.39, 5.35, 5.37, 5.24, 5.26, 5.08, 4.57, 4.44, 4.5, 4.56, 4.52, 4.59, 4.66, 4.67, 4.66, 4.72, 4.84, 4.81, 4.84, 4.88, 4.89, 4.82, 4.74, 4.84, 4.79, 4.63]]


### 编码哈密顿量

这里我们将式（2）中的二进制变量用式（3）替换，从而构建哈密顿量 $H_C$。

在编码哈密顿量的过程中，首先需要计算各股票之间的相似矩阵 $\rho$。

In [4]:
rho = data.get_similarity_matrix()

根据计算的相似矩阵和给定的参数构建哈密顿量。

In [5]:
q = 2 # 指数组合中需要的股票数目
penalty = num_assets # 惩罚参数：不小于可投资股票的数目
hamiltonian = portfolio_diversification_hamiltonian(penalty, rho, q)

### 计算损失函数

调用量桨内置的 [`complex entangled layer`](https://qml.baidu.com/api/paddle_quantum.circuit.uansatz.html) 构造参数化量子电路。该电路会返回一个输出态 $|\vec{\theta}\rangle$，由此输出态，我们可以定义投资组合分散化问题在经典-量子混合模型下的损失函数：

$$
L(\vec{\theta}) =  \langle\vec{\theta}|H_C|\vec{\theta}\rangle.
\tag{4}
$$

之后我们利用经典的优化算法寻找最优参数 $\vec{\theta}^*$。下面的代码给出了通过量桨和飞桨搭建网络的过程。

In [6]:
class PDNet(paddle.nn.Layer):

    def __init__(self, n, p, dtype="float64"):
        super(PDNet, self).__init__()

        self.p = p
        self.num_qubits = n * (n+1)
        self.theta = self.create_parameter(shape=[self.p, self.num_qubits, 3],
            default_initializer=paddle.nn.initializer.Uniform(low=0, high=2 * np.pi),
            dtype=dtype, is_bias=False)
        # print(self.theta)

    def forward(self, hamiltonian):
        """
        前向传播
        """
        cir = UAnsatz(self.num_qubits)
        cir.complex_entangled_layer(self.theta, self.p)
        cir.run_state_vector()
        loss = cir.expecval(hamiltonian)

        return loss, cir

### 训练量子神经网络

定义好了量子神经网络后，我们使用梯度下降的方法来更新其中的参数，使得式（4）的期望值最小。

In [7]:
SEED = 1100   # 随机数种子
p = 2       # 量子电路的层数
ITR = 150    # 迭代次数
LR = 0.4     # 梯度下降优化速率 

使用飞桨，优化上面定义的网络。

In [8]:
# 比特数量
n = len(rho)

# 固定随机数种子
paddle.seed(SEED)

# 定义量子神经网络
net = PDNet(n, p)

# 使用 Adam 优化器
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

# 梯度下降优化循环
for itr in range(1, ITR + 1):
    loss, cir = net(hamiltonian)
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()
    if itr % 10 == 0:
        print("循环数:", itr, "    损失:", "%.4f"% loss.numpy())

循环数: 10     损失: 7.7804
循环数: 20     损失: 5.4414
循环数: 30     损失: 3.6022
循环数: 40     损失: 3.2910
循环数: 50     损失: 1.9358
循环数: 60     损失: 0.3872
循环数: 70     损失: 0.1344
循环数: 80     损失: 0.0774
循环数: 90     损失: 0.0122
循环数: 100     损失: 0.0068
循环数: 110     损失: -0.0001
循环数: 120     损失: -0.0019
循环数: 130     损失: -0.0025
循环数: 140     损失: -0.0028
循环数: 150     损失: -0.0028


### 解码量子答案

当调用优化器求得损失函数的最小值以及相对应的一组参数 $\vec{\theta}^*$后，为了进一步求得投资组合优化问题的近似解，需要从电路输出的量子态 $|\vec{\theta}^*\rangle$ 中解码出经典优化问题的答案。物理上，解码量子态需要对量子态进行测量，然后统计测量结果的概率分布：

$$
p(z) = |\langle z|\vec{\theta}^*\rangle|^2.
\tag{6}
$$

在量子参数化电路表达能力足够的情况下，某个比特串出现的概率越大，意味着其是投资组合优化问题最优解的可能性越大。

量桨提供了查看参数化量子电路输出状态的测量结果概率分布的函数。

In [14]:
# 模拟重复测量电路输出态 2048 次
prob_measure = cir.measure(shots=2048)
investment = max(prob_measure, key=prob_measure.get)
print("利用哈密顿量找到的解的比特串形式：", investment)

利用哈密顿量找到的解的比特串形式： 100001001101


我们的测量结果是表示投资组合分散化问题答案的比特串：如上文结果 ``100001001101``，我们一共有 $n = 3$ 支可投资股票，选择两只到指数组合中。前 $n^2 = 9$ 位 ``100001001`` 代表 $x_{ij}$，每 $3$ 位为一组，第一组 ``100`` 中第一位为 $1$，代表它被划作一类。第二组 ``001`` 和第三组 ``001`` 中第三位被置为 $1$，代表它们被划为一类。同时，第一组和第三组 $1$ 出现的位置符合 $i = j$，即这两支股票为最能代表各自类的股票。另外，可以看出 $1$ 出现的位置是 $j = 1$ 和 $j = 3$，即两个位置可能为 $1$，这和我们预设的指数组合中有两只股票是对应的。同时，后 $3$ 位为 ``101``，代表 $y_j$, 表示第一支股票和第三支股票被选中放入指数组合中。通过上述说明，可以看出我们求解得到的结果是一个有效解。如果最后的结果不是上述这种有效解，读者依然可以通过调整参数化量子电路的参数值，来获得更好的训练效果。

### 结语

在本教程中，我们主要讨论了分散化投资中如何对可投资项目进行分类，以及如何挑选具有代表性的到我们的投资组合中来。在本问题中，每个投资项目都需要 $n$ 位量子比特来表示分类，$1$ 位量子比特表示是否被选中。受量子比特数目的限制，目前能够处理的投资项目数还比较少。

_______

## 参考文献

[1] Orus, Roman, Samuel Mugel, and Enrique Lizaso. "Quantum computing for finance: Overview and prospects." [Reviews in Physics 4 (2019): 100028.](https://arxiv.org/abs/1807.03890)

[2] Egger, Daniel J., et al. "Quantum computing for Finance: state of the art and future prospects." [IEEE Transactions on Quantum Engineering (2020).](https://arxiv.org/abs/2006.14510)